In [ ]:
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install seqeval

In [ ]:
import utils
from utils import compute_metrics
import datasets
from datasets import load_from_disk
from transformers import DistilBertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
import json


In [ ]:
# Defining model
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-cased", num_labels=25)

In [ ]:
train_dataset = load_from_disk('/content/drive/MyDrive/Colab Notebooks/underwriteme-data/combined_train_dataset')
print(train_dataset)
test_dataset = load_from_disk('/content/drive/MyDrive/Colab Notebooks/underwriteme-data/test_dataset')
print(test_dataset)

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 27454
})
Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 6223
})


In [ ]:
# Define the mapping from NER label to integer
label_to_id = {
    'O': 0,
    'B-Investigation': 1,
    'I-Investigation': 2,
    'B-Treatment': 3,
    'I-Treatment': 4,
    'B-Condition': 5,
    'I-Condition': 6,
    'B-Behaviour': 7,
    'I-Behaviour': 8,
    'B-Date': 9,
    'I-Date': 10,
    'B-Unit': 11,
    'I-Unit': 12,
    'B-Sign': 13,
    'I-Sign': 14,
    'B-Symptom': 15,
    'I-Symptom': 16,
    'B-Drug': 17,
    'I-Drug': 18,
    'B-LabResult': 19,
    'I-LabResult': 20,
    'B-HealthcareProvider': 21,
    'I-HealthcareProvider': 22,
    'B-LabTest': 23,
    'I-LabTest': 24,
}

id_to_label = {id: label for label, id in label_to_id.items()}

In [ ]:
# Define training args

args = TrainingArguments(
"test-ner2",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=3,
weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = datasets.load_metric("seqeval")

trainer = Trainer(
   model,
   args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

<ipython-input-5-70a41a89b401>:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerat

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.522200,0.981448,0.541667,0.657444,0.593966,0.758797
2,0.659400,0.287756,0.752183,0.840118,0.793722,0.945016
3,0.232800,0.114534,0.912801,0.948407,0.930264,0.982460
4,0.160600,0.066146,0.974733,0.984239,0.979463,0.991464
5,0.106900,0.055316,0.978960,0.986620,0.982775,0.992379


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=4290, training_loss=0.6399908239191229, metrics={'train_runtime': 694.8297, 'train_samples_per_second': 197.559, 'train_steps_per_second': 6.174, 'total_flos': 972931510765212.0, 'train_loss': 0.6399908239191229, 'epoch': 5.0})

In [ ]:
## Save model
model.save_pretrained("ner_model2")

## Save tokenizer
tokenizer.save_pretrained("ner_tokenizer2")

('ner_tokenizer_final/tokenizer_config.json',
 'ner_tokenizer_final/special_tokens_map.json',
 'ner_tokenizer_final/vocab.txt',
 'ner_tokenizer_final/added_tokens.json',
 'ner_tokenizer_final/tokenizer.json')

In [ ]:
# Write label-id into config file
config = json.load(open("ner_model2/config.json"))
config["id2label"] = id_to_label
config["label2id"] = label_to_id
json.dump(config, open("ner_model2/config.json","w"))